In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path
from random import seed
seed(252)

In [2]:
df = pd.read_csv(Path('./data/20201212.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],Rank your movie choices. You don't have to do a full ranking. [Fifth Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice],What theme should we explore next week? [Sixth Choice]
0,12/8/2020 17:37:11,Fireworks (1997),The Wild Goose Lake (2019),Tykho Moon (1996),Dick Tracy (1990),NaN,ZERO,"BEAUTIFUL FILM, UGLY SUBJECT MATTER","DAD, I'M IN JAIL",PORTUGAL,[Default to Nicolas Cage],DIRTY SOUTH
1,12/8/2020 17:42:14,You Were Never Really Here (2017),Fireworks (1997),The Wild Goose Lake (2019),Tykho Moon (1996),Dick Tracy (1990),"BEAUTIFUL FILM, UGLY SUBJECT MATTER","DAD, I'M IN JAIL",ZERO,PORTUGAL,DIRTY SOUTH,[Default to Nicolas Cage]
2,12/8/2020 18:52:17,The Wild Goose Lake (2019),You Were Never Really Here (2017),Fireworks (1997),Tykho Moon (1996),NaN,DIRTY SOUTH,PORTUGAL,ZERO,"DAD, I'M IN JAIL",[Default to Nicolas Cage],NaN
3,12/8/2020 20:50:28,Tykho Moon (1996),You Were Never Really Here (2017),Dick Tracy (1990),Fireworks (1997),The Wild Goose Lake (2019),PORTUGAL,DIRTY SOUTH,[Default to Nicolas Cage],"BEAUTIFUL FILM, UGLY SUBJECT MATTER","DAD, I'M IN JAIL",ZERO
4,12/9/2020 20:35:46,You Were Never Really Here (2017),The Wild Goose Lake (2019),Tykho Moon (1996),Fireworks (1997),Dick Tracy (1990),"BEAUTIFUL FILM, UGLY SUBJECT MATTER",DIRTY SOUTH,PORTUGAL,ZERO,"DAD, I'M IN JAIL",[Default to Nicolas Cage]
5,12/10/2020 8:51:20,You Were Never Really Here (2017),Dick Tracy (1990),The Wild Goose Lake (2019),Fireworks (1997),Tykho Moon (1996),DIRTY SOUTH,"DAD, I'M IN JAIL","BEAUTIFUL FILM, UGLY SUBJECT MATTER",[Default to Nicolas Cage],ZERO,PORTUGAL


In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['You Were Never Really Here (2017)', 'Fireworks (1997)',
       'The Wild Goose Lake (2019)', 'Tykho Moon (1996)',
       'Dick Tracy (1990)'], dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'You Were Never Really Here (2017)': <Candidate('You Were Never Really Here (2017)')>,
 'Fireworks (1997)': <Candidate('Fireworks (1997)')>,
 'The Wild Goose Lake (2019)': <Candidate('The Wild Goose Lake (2019)')>,
 'Tykho Moon (1996)': <Candidate('Tykho Moon (1996)')>,
 'Dick Tracy (1990)': <Candidate('Dick Tracy (1990)')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(Fireworks (1997), The Wild Goose Lake (2019), Tykho Moon (1996), Dick Tracy (1990))>,
 <Ballot(You Were Never Really Here (2017), Fireworks (1997), The Wild Goose Lake (2019), Tykho Moon (1996), Dick Tracy (1990))>,
 <Ballot(The Wild Goose Lake (2019), You Were Never Really Here (2017), Fireworks (1997), Tykho Moon (1996))>,
 <Ballot(Tykho Moon (1996), You Were Never Really Here (2017), Dick Tracy (1990), Fireworks (1997), The Wild Goose Lake (2019))>,
 <Ballot(You Were Never Really Here (2017), The Wild Goose Lake (2019), Tykho Moon (1996), Fireworks (1997), Dick Tracy (1990))>,
 <Ballot(You Were Never Really Here (2017), Dick Tracy (1990), The Wild Goose Lake (2019), Fireworks (1997), Tykho Moon (1996))>]

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

ROUND 1
Candidate                            Votes  Status
---------------------------------  -------  --------
You Were Never Really Here (2017)        3  Hopeful
Fireworks (1997)                         1  Hopeful
The Wild Goose Lake (2019)               1  Rejected
Tykho Moon (1996)                        1  Rejected
Dick Tracy (1990)                        0  Rejected

FINAL RESULT
Candidate                            Votes  Status
---------------------------------  -------  --------
You Were Never Really Here (2017)        5  Elected
Fireworks (1997)                         1  Rejected
The Wild Goose Lake (2019)               0  Rejected
Tykho Moon (1996)                        0  Rejected
Dick Tracy (1990)                        0  Rejected



In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                              Votes  Status
-----------------------------------  -------  --------
DIRTY SOUTH                                2  Hopeful
BEAUTIFUL FILM, UGLY SUBJECT MATTER        2  Hopeful
PORTUGAL                                   1  Rejected
ZERO                                       1  Rejected
DAD, I'M IN JAIL                           0  Rejected
[Default to Nicolas Cage]                  0  Rejected

FINAL RESULT
Candidate                              Votes  Status
-----------------------------------  -------  --------
DIRTY SOUTH                                3  Elected
BEAUTIFUL FILM, UGLY SUBJECT MATTER        3  Rejected
PORTUGAL                                   0  Rejected
ZERO                                       0  Rejected
DAD, I'M IN JAIL                           0  Rejected
[Default to Nicolas Cage]                  0  Rejected

